In [1]:
import numpy as np
from ipywidgets import IntSlider

import plotlymath as plt
from myutils import interact, latex


In [8]:
plt.set_defaults(margin=(25,0,40,50))

In [11]:
A = np.array(((0.47, 0.42), (0.24, 0.85)))

In [16]:
def blackbear_interactive_single(A):
    xmax, ymax = (850, 850)
    years = 30
    solution = [np.array((500, 250), dtype=float)]
    for n in range(years):
        solution.append(A @ solution[-1])

    figure, plot = plt.make_figure(widget=True)
    figure.layout.update(width=750, height=600)
    figure.layout.xaxis.domain = (0, 0.75)
    plot.axes_labels("Juveniles", "Adults")
    plot.axes_ranges((0, xmax), (0, ymax), scale=(1, 1))

    @interact(n=IntSlider(min=0, max=years, value=0))
    def update(n):
        label = latex((f"J({n})", f"A({n})")) + " = " + latex(solution[n], 1)
        with figure.batch_update():
            plot.bears = plt.points(solution[:n+1])
            plot.label = plt.text(f"${label}$", (0.77, 0.5), paper=True, font_size=24)

    return figure

blackbear_interactive_single(A)


interactive(children=(IntSlider(value=0, description='n', max=30), Output()), _dom_classes=('widget-interact',…

FigureWidget({
    'data': [{'marker': {'size': 8},
              'mode': 'markers',
              'type': 'sc…

In [17]:
def blackbear_interactive_many(A):
    xmax, ymax = (850, 850)
    years = 30
    solutions = {
        "blue":       [np.array((500, 250), dtype=float)], 
        "darkorange": [np.array((500,  50), dtype=float)], 
        "purple":     [np.array((150, 500), dtype=float)], 
        "magenta":    [np.array(( 50, 700), dtype=float)], 
    }
    for solution in solutions.values():
        for t in range(years):
            solution.append(A @ solution[-1])

    figure, plot = plt.make_figure(widget=True)
    figure.layout.update(width=750, height=600)
    figure.layout.xaxis.domain = (0, 0.75)
    plot.axes_labels("Juveniles", "Adults")
    plot.axes_ranges((0, xmax), (0, ymax), scale=(1, 1))
    length = np.linalg.norm((xmax, ymax))
    evalues, evectors = np.linalg.eig(A)
    eigenlines = []
    for evalue, evector in zip(evalues, evectors.transpose()):
        eigenline = np.array([evector * -length, evector * length])
        color = "darkred" if abs(evalue) < 1 else "darkgreen"
        eigenlines.append(plt.lines(eigenline, color=color, opacity=0.6, line_width=4, 
                name="Eigenline", legendgroup="eigenlines", visible="legendonly"))

    controls = {color: IntSlider(min=0, max=years, value=0) for color in solutions}
    @interact(**controls)
    def update(**controls):
        with figure.batch_update():
            for i, (color, solution) in enumerate(solutions.items()):
                n = controls[color]
                label = latex((f"J({n})", f"A({n})")) + " = " + latex(solution[n], 1)
                setattr(plot, f"{color}pts", plt.points(solution[:n+1], 
                        color=color, mode="markers+lines", showlegend=False))
                setattr(plot, f"{color}label", plt.text(f"${label}$", (0.77, 0.85-0.2*i), 
                        color=color, paper=True, size=24))

    update(**{color: 0 for color in solutions})
    plot.eigenlines = eigenlines
    return figure

blackbear_interactive_many(A)


interactive(children=(IntSlider(value=0, description='blue', max=30), IntSlider(value=0, description='darkoran…

FigureWidget({
    'data': [{'marker': {'color': 'blue', 'size': 8},
              'mode': 'markers+lines',
  …

In [14]:
evalues, evectors = np.linalg.eig(A)
for evalue, evector in zip(evalues, evectors.transpose()):
    vector = np.round(-5*evector, 4)
    print(f"Eigenvalue λ = {evalue:.2f} with eigenvector {tuple(vector)}")


Eigenvalue λ = 0.29 with eigenvector (4.5957, -1.9696)
Eigenvalue λ = 1.03 with eigenvector (3.0, 4.0)


# A two-stage black bear population model

**Assumptions:** 
- The population is subdivided into two types of bears: juveniles ($J$) and adults ($A$). 
- Each year, on average, $42\%$ of adults give birth to a cub. 
- Each year, $24\%$ of juveniles reach adulthood. 
- Each year, $15\%$ of adult bears die, and $29\%$ of juvenile bears die. 

**Resulting model:** 

<!-- $$ \begin{bmatrix} J(n+1) \\ A(n+1) \end{bmatrix} = \begin{bmatrix} 0.47 J(n) + 0.42 A(n) \\ 0.24 J(n) + 0.85 A(n) \end{bmatrix} $$ -->
$$ \begin{cases} J(n+1) = 0.47 J(n) + 0.42 A(n) \\ A(n+1) = 0.24 J(n) + 0.85 A(n) \end{cases} $$
or, in matrix form: 
$$ \begin{bmatrix} J(n+1) \\ A(n+1) \end{bmatrix} = \begin{pmatrix} 0.47 & 0.42 \\ 0.24 & 0.85 \end{pmatrix} \begin{bmatrix} J(n) \\ A(n) \end{bmatrix} $$
